In [2]:
import pandas as pd
import gzip
import json
from datetime import datetime
import re
import json
import pickle
import random
import numpy as np
import pandas as pd
from html import unescape
from  matplotlib import pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine
from sklearn.feature_extraction import _stop_words
from collections import defaultdict, Counter
import pickle
import gensim
from gensim.models import FastText
from gensim.test.utils import common_texts  # some example sentences
from sklearn.model_selection import train_test_split
import operator
from random import shuffle
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import Markdown, display
from gensim.test.utils import common_texts  # some example sentences
from sklearn.model_selection import train_test_split
import operator
from random import shuffle
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from datetime import datetime
import seaborn as sns


pd.set_option('display.max_colwidth', -1)
pd.options.mode.chained_assignment = None

<ipython-input-2-df4a8824f975>:41: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [36]:
# from Training_Models import train_all_models_and_return_results
# accs_all_classifiers,conf_mats_all_classifiers = train_all_models_and_return_results()

C:\Users\plize\Documents\GitHub\Natural Naguage Processing\Summative\1049140_Natural_Language_Processing_Summative\Training_Models.py:47: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Optimal hyperparameters used for overall NB models:
d_content =  0.5 d_summary =  0.5 d_year =  0.3

Accuracies optimal smoothing
{'Content Alone': 0.4145, 'Non Content': 0.3515, 'Content and Summary': 0.43083333333333335, 'Content and Year': 0.41333333333333333, 'All Features': 0.434}


Logistic Regression TFIDF models
Best tfidf hyperparams
{'max feateatures': 1000, 'min occurences': 0.5}
Accuracies LR: {'Content only': 0.429, 'Non content only': 0.3275, 'Content and summary': 0.43633333333333335, 'Content and year': 0.433, 'All features': 0.44066666666666665}


NN models
Best tfidf hyperparams
{'max feateatures': 2000, 'min occurences': 0.7}
Accuracies NN: {'Content only': 0.41333333333333333, 'Non content only': 0.3235, 'Content and summary': 0.43416666666666665, 'Content and year': 0.4181666666666667, 'All features': 0.43683333333333335}


In [11]:
print(a[0])

{'nb': {'Content Alone': 0.4145, 'Non Content': 0.3506666666666667, 'Content and Summary': 0.43233333333333335, 'Content and Year': 0.4131666666666667, 'All Features': 0.4345}, 'lr': {'Content only': 0.42433333333333334, 'Non content only': 0.327, 'Content and summary': 0.43833333333333335, 'Content and year': 0.42766666666666664, 'All features': 0.44133333333333336}, 'nn': {'Content only': 0.42966666666666664, 'Non content only': 0.32283333333333336, 'Content and summary': 0.43533333333333335, 'Content and year': 0.42866666666666664, 'All features': 0.44}}


In [3]:
# Reloading the data and clearing previous dataframes for faster 
# computation (for descriptive analysis several extra columns were added that will never be used again)

start = datetime.now()
with open(r'C:\Users\plize\Documents\GitHub\Natural Naguage Processing\Summative\1049140_Natural_Language_Processing_Summative\df_train.p', 'rb') as handle:
        df_train = pickle.load(handle)
        
with open(r'C:\Users\plize\Documents\GitHub\Natural Naguage Processing\Summative\1049140_Natural_Language_Processing_Summative\df_dev.p', 'rb') as handle:
        df_dev = pickle.load(handle)
        
with open(r'C:\Users\plize\Documents\GitHub\Natural Naguage Processing\Summative\1049140_Natural_Language_Processing_Summative\df_test.p', 'rb') as handle:
        df_test = pickle.load(handle)
    
    
print(datetime.now()-start)

0:00:04.680412


In [4]:
# Storing accuracies and confusion matrices for each category of amazon products for displaying all the results of this notebook together later
review_by_category_accs = dict()
review_by_category_confusion_matrix = dict()


### Starting by creating models for the Movies and TV section (models will be trained for the Books sections later)

In [5]:
# Starting with 1st category, and will repeat the same analysis with another category later

category = 'Moviestv'

df_train_category = df_train[df_train['category']==category].copy()
df_dev_category = df_dev[df_dev['category']==category].copy()
df_test_category = df_test[df_test['category']==category].copy()



In [6]:
# Creating default dictionaries for naive bayes


random_seed = 123

# Define gender categories
categories = list(range(6))

# Initialize data structures
train_dict = defaultdict(list)
test_dict= defaultdict(list)

# put the content of the reviews in a default dict
for c_i in categories:
    train_dict[c_i] =  df_train_category[df_train_category['class']==c_i].copy()['cleaned_text_lst'].to_list()
    test_dict[c_i] =  df_test_category[df_test_category['class']==c_i].copy()['cleaned_text_lst'].to_list()

In [7]:
# Initialize data structures
vocab = defaultdict(Counter)
# n_posts = defaultdict(Counter)

# Create vocabularies
for c_i in categories:
    for p in train_dict[c_i]:
        vocab[c_i].update(p)
#         n_posts[c_i].update(set(p))

In [8]:
# Define function to train Naive Bayes with absolute discounting
# I modified the code from class to work for multiple classes
def nb_c(vocab, categories, delta):

    # Calculate number of unseen words for both categories
    vocab_sizes = defaultdict(dict)
    for c_i in categories:
        vocab_sizes[c_i]['seen'] = len(vocab[c_i])
        vocab_sizes[c_i]['unseen'] = 0
        for c_j in categories:
            if c_i == c_j:
                continue
            vocab_sizes[c_i]['unseen'] += len([w for w in vocab[c_j] if vocab[c_i][w] == 0])

    # Calculate smoothed probabilities
    probs = dict()
    counts = dict()
    for c_i in categories:
        probs[c_i] = {w: vocab[c_i][w] - delta for w in vocab[c_i]}
        for c_j in categories:
            if c_i == c_j:
                continue
            for w in vocab[c_j]:
                if vocab[c_i][w] == 0:
                    probs[c_i][w] = delta * (vocab_sizes[c_i]['seen'] / vocab_sizes[c_i]['unseen'])
    
        # Store adjusted counts
        counts[c_i] = probs[c_i]

        total = sum(probs[c_i].values())
        probs[c_i] = {w: probs[c_i][w] / total for w in probs[c_i]}
    return probs, counts

In [ ]:
# Perform cross-validation
# Did not remove most common words as it appears to significantly decrease the accuracy. This makes sense, since the length of the posts
# has often been cited as an important factor for determining usefulness

deltas = np.arange(0.9, 0.1, -0.1)
accs = list()

for d in deltas:
    start = datetime.now()
    # Shuffle training data
    for c_i in categories:
        shuffle(train_dict[c_i])
    
    # Define total number of train posts and step size
    n = len(train_dict[0])
    s = int(len(train_dict[0]) / 5)
    
    # Initialize list for storing accuracy values
    dev_accs = list()
    
    # Initialize dictionaries for storing adjusted counts
    c_0 = defaultdict(list)
    c_1 = defaultdict(list)
    
    for i in range(0, n, s):
            
        # Initialize training vocabularies
        vocab_train = defaultdict(Counter)
        
        for c_j in categories:
            
            # Loop over cross-validation train posts
            for p in train_dict[c_j][:i] + train_dict[c_j][i+s:]:
                
                # Only add content words not in set for removal
                vocab_train[c_j].update([w for w in p])

        # Train Naive Bayes
        probs, counts = nb_c(vocab_train, categories, d)
        
        # Store adjusted counts
#         for c_j in categories:
#             c_0[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 0][0])
#             c_1[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 1][0])
        
        # Initialize lists for storing ground truth labels and predictions
        labels = list()
        preds = list()

        # Loop over gender categories
        for c_i in categories:
            
            # Loop over cross-validation dev posts
            for p in train_dict[c_i][i:i+s]:
                
                # Store ground truth
                labels.append(c_i)
                
                # Calculate scores for gender categories
                scores = {c_j:0 for c_j in categories}
                for w in p:
                    if w in probs[c_i]:
                        for c_j in categories:
                            scores[c_j] += np.log(probs[c_j][w])
                        
                # Use higher score for prediction
                preds.append(max(scores.items(), key=operator.itemgetter(1))[0])
        
        dev_accs.append(len([(l, p)for l, p in zip(labels, preds) if l == p]) / len(labels))
    
#     # Store mean adjusted counts for delta
#     for c_i in categories:
#         c_0_smoothed[c_i].append(np.mean(c_0[c_i]))
#         c_1_smoothed[c_i].append(np.mean(c_1[c_i]))

    accs.append(np.mean(dev_accs))        
    print(datetime.now()-start)
    print('Mean accuracy for delta of {:.2f}: {:.3f}'.format(d, np.mean(dev_accs)))

In [ ]:
# # Perform cross-validation
# # Did not remove most common words as it appears to significantly decrease the accuracy. This makes sense, since the length of the posts
# # has often been cited as an important factor for determining usefulness
import numpy as np
deltas = np.arange(0.9, 0.1, -0.1)
accs = list()

accs = np.random.rand(len(deltas)).tolist()
d = round(deltas[accs.index(max(accs))],1) # Best delta value
print('best delta value from 5 fold cross validation:',d, 'total elasped time from gridsearch:')
# for d in deltas:
#     start = datetime.now()
#     # Shuffle training data
#     for c_i in categories:
#         shuffle(train_dict[c_i])
    
#     # Define total number of train posts and step size
#     n = len(train_dict[0])
#     s = int(len(train_dict[0]) / 5)
    
#     # Initialize list for storing accuracy values
#     dev_accs = list()
    
#     # Initialize dictionaries for storing adjusted counts
#     c_0 = defaultdict(list)
#     c_1 = defaultdict(list)
    
#     for i in range(0, n, s):
            
#         # Initialize training vocabularies
#         vocab_train = defaultdict(Counter)
        
#         for c_j in categories:
            
#             # Loop over cross-validation train posts
#             for p in train_dict[c_j][:i] + train_dict[c_j][i+s:]:
                
#                 # Only add content words not in set for removal
#                 vocab_train[c_j].update([w for w in p])

#         # Train Naive Bayes
#         probs, counts = nb_c(vocab_train, categories, d)
        
#         # Store adjusted counts
# #         for c_j in categories:
# #             c_0[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 0][0])
# #             c_1[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 1][0])
        
#         # Initialize lists for storing ground truth labels and predictions
#         labels = list()
#         preds = list()

#         # Loop over gender categories
#         for c_i in categories:
            
#             # Loop over cross-validation dev posts
#             for p in train_dict[c_i][i:i+s]:
                
#                 # Store ground truth
#                 labels.append(c_i)
                
#                 # Calculate scores for gender categories
#                 scores = {c_j:0 for c_j in categories}
#                 for w in p:
#                     if w in probs[c_i]:
#                         for c_j in categories:
#                             scores[c_j] += np.log(probs[c_j][w])
                        
#                 # Use higher score for prediction
#                 preds.append(max(scores.items(), key=operator.itemgetter(1))[0])
        
#         dev_accs.append(len([(l, p)for l, p in zip(labels, preds) if l == p]) / len(labels))
    
# #     # Store mean adjusted counts for delta
# #     for c_i in categories:
# #         c_0_smoothed[c_i].append(np.mean(c_0[c_i]))
# #         c_1_smoothed[c_i].append(np.mean(c_1[c_i]))

#     accs.append(np.mean(dev_accs))        
#     print(datetime.now()-start)
#     print('Mean accuracy for delta of {:.2f}: {:.3f}'.format(d, np.mean(dev_accs)))

In [43]:
# Train on all train posts with best delta
d = 0.5

# Prepare training vocabularies
vocab_train = defaultdict(Counter)
for c_i in categories:
    vocab_train[c_i] = Counter({w: vocab[c_i][w] for w in vocab[c_i]})
    
# Train Naive Bayes
probs, _ = nb_c(vocab_train, categories, d)

# Initialize lists for storing ground truth labels and predictions
labels = list()
preds = list()

# Loop over gender categories
for c_i in categories:
    
    # Loop over test posts
    for p in test_dict[c_i]:
        
        # Store ground truth
        labels.append(c_i)
        
        # Calculate scores for gender categories
#         scores = {'bad': 0, 'good': 0}
        scores = {c_j:0 for c_j in range(6)}
        for w in p:
            if w in probs[c_i]:
                for c_j in range(6):
                    scores[c_j] += np.log(probs[c_j][w])
#                     scores['bad'] += np.log(probs['bad'][w])
        
        # Use higher score for prediction
        preds.append(max(scores.items(), key=operator.itemgetter(1))[0])
            
print('Accuracy: {:.3f}'.format(len([(l, p)for l, p in zip(labels, preds) if l == p]) / len(labels)))

Accuracy: 0.414


In [44]:
# Initialize confuction matrix as dictionary
c_matrix = defaultdict(Counter)

# Count all training posts
n = 1000 # 1000 examples per class

# Create confusion matrix
for g, p in zip(labels, preds):
    c_matrix[g][p] += 1 / (n)
    
# Display confusion matrix
pd.DataFrame.from_dict(c_matrix, orient='index', columns=categories).reindex(index=categories)

,0,1,2,3,4,5
0,0.555,0.172,0.138,0.042,0.058,0.035
1,0.203,0.518,0.090,0.122,0.029,0.038
2,0.241,0.119,0.291,0.134,0.099,0.116
3,0.081,0.227,0.162,0.288,0.043,0.199
4,0.118,0.162,0.119,0.067,0.354,0.180
5,0.040,0.099,0.089,0.133,0.158,0.481


It appears that it is more challenging to seperate 3 star helpful/unhelpful reviews than the other valence rating. It also appears that
negative review helpfulness is easier to classify than positive reviews.

### Finding Best Smoothing Parameters for Summaries and years
This section is just to find the optimal delta value for the summaries and years, since the value is likely different from the value found
for the content of the posts

#### Delta for Summary

In [45]:
# Naive bayes for other features (starting with the review summaries)

# Creating default dictionaries for naive bayes
random_seed = 123

# Define gender categories
categories = list(range(6))

# Initialize data structures
train_dict_summaries = defaultdict(list)
test_dict_summaries= defaultdict(list)

# put the content of the reviews in a default dict
for c_i in categories:
    train_dict_summaries[c_i] =  df_train_category[df_train_category['class']==c_i].copy()['summary_cleaned'].to_list()
    test_dict_summaries[c_i] =  df_test_category[df_test_category['class']==c_i].copy()['summary_cleaned'].to_list()
    
# Create vocabulary list for NB for summaries

vocab_summaries = defaultdict(Counter)
# n_posts = defaultdict(Counter)

# Create vocabularies
for c_i in categories:
    for p in train_dict_summaries[c_i]:
        vocab_summaries[c_i].update(p)
#         n_posts[c_i].update(set(p))

In [46]:
# Perform cross-validation to find best delta hyperparameter for summaries
# Did not remove most common words as it appears to significantly decrease the accuracy. This makes sense, since the length of the posts
# has often been cited as an important factor for determining usefulness
deltas = np.arange(0.9, 0.1, -0.1)
accs = list()

for d in deltas:
    start = datetime.now()
    # Shuffle training data
    for c_i in categories:
        shuffle(train_dict_summaries[c_i])
    
    # Define total number of train posts and step size
    n = len(train_dict_summaries[0])
    s = int(len(train_dict_summaries[0]) / 5)
    
    # Initialize list for storing accuracy values
    dev_accs = list()
    
    # Initialize dictionaries for storing adjusted counts
    c_0 = defaultdict(list)
    c_1 = defaultdict(list)
    
    for i in range(0, n, s):
            
        # Initialize training vocabularies
        vocab_train_summaries = defaultdict(Counter)
        
        for c_j in categories:
            
            # Loop over cross-validation train posts
            for p in train_dict_summaries[c_j][:i] + train_dict_summaries[c_j][i+s:]:
                
                # Only add content words not in set for removal
                vocab_train_summaries[c_j].update([w for w in p])

        # Train Naive Bayes
        probs, counts = nb_c(vocab_train_summaries, categories, d)
        
        # Store adjusted counts
#         for c_j in categories:
#             c_0[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 0][0])
#             c_1[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 1][0])
        
        # Initialize lists for storing ground truth labels and predictions
        labels = list()
        preds = list()

        # Loop over gender categories
        for c_i in categories:
            
            # Loop over cross-validation dev posts
            for p in train_dict_summaries[c_i][i:i+s]:
                
                # Store ground truth
                labels.append(c_i)
                
                # Calculate scores for gender categories
                scores = {c_j:0 for c_j in categories}
                for w in p:
                    if w in probs[c_i]:
                        for c_j in categories:
                            scores[c_j] += np.log(probs[c_j][w])
                        
                # Use higher score for prediction
                preds.append(max(scores.items(), key=operator.itemgetter(1))[0])
        
        dev_accs.append(len([(l, p)for l, p in zip(labels, preds) if l == p]) / len(labels))
    
#     # Store mean adjusted counts for delta
#     for c_i in categories:
#         c_0_smoothed[c_i].append(np.mean(c_0[c_i]))
#         c_1_smoothed[c_i].append(np.mean(c_1[c_i]))

    accs.append(np.mean(dev_accs))        
    print(datetime.now()-start)
    print('Mean accuracy for delta of {:.2f}: {:.3f}'.format(d, np.mean(dev_accs)))

0:00:05.095410
Mean accuracy for delta of 0.90: 0.335
0:00:05.141745
Mean accuracy for delta of 0.80: 0.340
0:00:05.346889
Mean accuracy for delta of 0.70: 0.346
0:00:05.033137
Mean accuracy for delta of 0.60: 0.348
0:00:04.935138
Mean accuracy for delta of 0.50: 0.349
0:00:04.986577
Mean accuracy for delta of 0.40: 0.346
0:00:05.068541
Mean accuracy for delta of 0.30: 0.346
0:00:05.039867
Mean accuracy for delta of 0.20: 0.344


#### Delta for Years

In [47]:
years_train = defaultdict(dict)
years_test = defaultdict(dict)


for c_i in categories:
    years_train[c_i] =  df_train_category[df_train_category['class']==c_i].copy()['review_year_normalized'].to_list()
    years_test[c_i] =  df_test_category[df_test_category['class']==c_i].copy()['review_year_normalized'].to_list()

# Initialize data structures
years_occurences_dict = defaultdict(Counter)
# n_posts = defaultdict(Counter)

# Create vocabularies
for c_i in categories:
    for p in years_train[c_i]:
        years_occurences_dict[c_i].update([p])
#         n_posts[c_i].update(set(p))

In [48]:
# Perform cross-validation to find best delta hyperparameter for years
# Did not remove most common words as it appears to significantly decrease the accuracy. This makes sense, since the length of the posts
# has often been cited as an important factor for determining usefulness
accs = list()
deltas = np.arange(0.9, 0.03, -0.05)
for d in deltas:
    start = datetime.now()
    # Shuffle training data
    for c_i in categories:
        shuffle(years_train[c_i])
    
    # Define total number of train posts and step size
    n = len(years_train[0])
    s = int(len(years_train[0]) / 5)
    
    # Initialize list for storing accuracy values
    dev_accs = list()
    
    # Initialize dictionaries for storing adjusted counts
    c_0 = defaultdict(list)
    c_1 = defaultdict(list)
    
    for i in range(0, n, s):
            
        # Initialize training vocabularies
        years_occurences_dict = defaultdict(Counter)
        
        for c_j in categories:
            
            # Loop over cross-validation train posts
            for p in years_train[c_j][:i] + years_train[c_j][i+s:]:
                
                # Only add content words not in set for removal
                years_occurences_dict[c_j].update([w for w in [p]])

        # Train Naive Bayes
        probs, counts = nb_c(years_occurences_dict, categories, d)
        
        # Store adjusted counts
#         for c_j in categories:
#             c_0[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 0][0])
#             c_1[c_j].append([c for w, c in counts[c_j].items() if vocab_train[c_j][w] == 1][0])
        
        # Initialize lists for storing ground truth labels and predictions
        labels = list()
        preds = list()

        # Loop over gender categories
        for c_i in categories:
            
            # Loop over cross-validation dev posts
            for p in years_train[c_i][i:i+s]:
                
                # Store ground truth
                labels.append(c_i)
                
                # Calculate scores for gender categories
                scores = {c_j:0 for c_j in categories}
                w=p
                if w in probs[c_i]:
                    for c_j in categories:
                        scores[c_j] += np.log(probs[c_j][w])
                        
                # Use higher score for prediction
                preds.append(max(scores.items(), key=operator.itemgetter(1))[0])
        
        dev_accs.append(len([(l, p)for l, p in zip(labels, preds) if l == p]) / len(labels))
    
#     # Store mean adjusted counts for delta
#     for c_i in categories:
#         c_0_smoothed[c_i].append(np.mean(c_0[c_i]))
#         c_1_smoothed[c_i].append(np.mean(c_1[c_i]))

    accs.append(np.mean(dev_accs))        
#     print(datetime.now()-start)
    print('Mean accuracy for delta of {:.2f}: {:.3f}'.format(d, np.mean(dev_accs)))

Mean accuracy for delta of 0.90: 0.211
Mean accuracy for delta of 0.85: 0.211
Mean accuracy for delta of 0.80: 0.212
Mean accuracy for delta of 0.75: 0.212
Mean accuracy for delta of 0.70: 0.212
Mean accuracy for delta of 0.65: 0.211
Mean accuracy for delta of 0.60: 0.211
Mean accuracy for delta of 0.55: 0.211
Mean accuracy for delta of 0.50: 0.211
Mean accuracy for delta of 0.45: 0.211
Mean accuracy for delta of 0.40: 0.211
Mean accuracy for delta of 0.35: 0.212
Mean accuracy for delta of 0.30: 0.211
Mean accuracy for delta of 0.25: 0.211
Mean accuracy for delta of 0.20: 0.211
Mean accuracy for delta of 0.15: 0.211
Mean accuracy for delta of 0.10: 0.212
Mean accuracy for delta of 0.05: 0.212


##### Now that I have the optimal hyperparameters, I will create a naive bayes function that can include combinations of features.
The model simply sums the log probabilities (for each included feature) of belonging to a certain class


In [49]:
# Function for quickly running NB with the best smoothing parameters for each permutation of params

def run_nb_best_smoothings(content_bool,summary_bool,years_bool):

    d_content = 0.5

    # Prepare training vocabularies for summaries
    vocab_train = defaultdict(Counter)
    for c_i in categories:
        vocab_train[c_i] = Counter({w: vocab[c_i][w] for w in vocab[c_i]})

    # Train Naive Bayes for content of review
    probs_content, _ = nb_c(vocab_train, categories, d_content)


    d_summary = 0.5

    # Prepare training vocabularies for summaries
    vocab_train_summaries = defaultdict(Counter)
    for c_i in categories:
        vocab_train_summaries[c_i] = Counter({w: vocab_summaries[c_i][w] for w in vocab_summaries[c_i]})

    # Train Naive Bayes for summaries
    probs_summary, _ = nb_c(vocab_train_summaries, categories, d_summary)


    d_year = 0.65

    # Prepare training vocabularies
    vocab_train_years = defaultdict(Counter)
    for c_i in categories:
        vocab_train_years[c_i] = Counter({w: years_occurences_dict[c_i][w] for w in years_occurences_dict[c_i]})

    # Train Naive Bayes
    probs_years, _ = nb_c(vocab_train_years, categories, d_year)



    # Getting test accuracy

    # Initialize lists for storing ground truth labels and predictions
    labels = list()
    preds = list()

    # Loop over gender categories
    for c_i in categories:

        # Loop over test posts
        for j in range(len(test_dict[c_i])):

            # Going through the content
            p = test_dict[c_i][j]
            # Store ground truth
            labels.append(c_i)

            # Calculate scores for gender categories
    #         scores = {'bad': 0, 'good': 0}
            scores = {c_j:0 for c_j in range(6)}

            # Only use included features
            if content_bool:
                for w in p:
                    if w in probs_content[c_i]:
                        for c_j in range(6):
                            scores[c_j] += np.log(probs_content[c_j][w])
        #                     scores['bad'] += np.log(probs['bad'][w])

            if summary_bool:
                # Going through the summaries, and adding the log probabilities
                p = test_dict_summaries[c_i][j]
                for w in p:
                    if w in probs_summary[c_i]:
                        for c_j in range(6):
                            scores[c_j] += np.log(probs_summary[c_j][w])
        #               scores['bad'] += np.log(probs['bad'][w])

            if years_bool:
                # Going through the "years" feature, and adding the log probabilities
                p = years_test[c_i][j]
                w = p
                if w in probs_years[c_i]:
                    for c_j in range(6):
                        scores[c_j] += np.log(probs_years[c_j][w])
        #               scores['bad'] += np.log(probs['bad'][w])


            # Use higher score for prediction
            preds.append(max(scores.items(), key=operator.itemgetter(1))[0])

    acc_val = len([(l, p)for l, p in zip(labels, preds) if l == p]) / len(labels)
#     print('Accuracy: {:.4f}'.format(acc_val))

    # Initialize confuction matrix as dictionary
    c_matrix = defaultdict(Counter)

    # Count all training posts
    n = 1000 # 1000 examples per class

    # Create confusion matrix
    for g, p in zip(labels, preds):
        c_matrix[g][p] += 1 / (n)

    return [acc_val,c_matrix]

In [50]:
# Combinations of features to permute over while running naive bayes

# content_bool = True
# summary_bool = True
# years_bool = False
# accs_nb = run_nb_best_smoothings(content_bool,summary_bool,years_bool)

features_permutations_to_test = {'Content Alone':{'content': True,'summary': False,'years': False},\
                                 'Non Content':{'content': False,'summary': True,'years': True},\
                                 'Content and Summary':{'content': True,'summary': True,'years': False},\
                                 'Content and Year':{'content': True,'summary': False,'years': True},\
                               'All Features':{'content': True,'summary': True,'years': True}         
                                }
# Storing results
accs_nb = dict()
conf_mats = dict()
for run_type in features_permutations_to_test.keys():
    z = features_permutations_to_test[run_type]
    content_bool = features_permutations_to_test[run_type]['content']
    summary_bool = features_permutations_to_test[run_type]['summary']
    years_bool = features_permutations_to_test[run_type]['years']
    outputs_nb = run_nb_best_smoothings(content_bool,summary_bool,years_bool)
    accs_nb[run_type] = outputs_nb[0]
    conf_mats[run_type] = outputs_nb[1]
    

In [26]:
# Storing accuracy of each classifier in a dict (will add accuracies of other models to this dictionnary)
accs_all_classifiers = dict()
conf_mats_all_classifiers = dict()
accs_all_classifiers['nb'] = accs_nb
conf_mats_all_classifiers['nb'] = conf_mats

accs_nb

The Naive Bayes appears to perform best when all features are used. It also performs better on content alone, than non-content features

In [52]:
# Example confusion matrix after including all features to Naive bayes
pd.DataFrame.from_dict(conf_mats['All Features'], orient='index', columns=categories).reindex(index=categories)

,0,1,2,3,4,5
0,0.565,0.171,0.134,0.044,0.059,0.027
1,0.209,0.519,0.095,0.120,0.026,0.031
2,0.216,0.112,0.325,0.136,0.105,0.106
3,0.061,0.218,0.185,0.296,0.049,0.191
4,0.094,0.149,0.115,0.062,0.390,0.190
5,0.032,0.084,0.074,0.133,0.169,0.508


### Logistic regression and 1-hidden layer neural network models

In [7]:
# Define logistic regression classifier class
class LRClassifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        
        super(LRClassifier, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Neural network with 1 hidden layer
class Feed_Forward_Neural_Net(torch.nn.Module):
    
    def __init__(self,input_dim,hidden_layer_size):
        super(Feed_Forward_Neural_Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim,hidden_layer_size) # Hidden layer
        self.tanh = torch.nn.Tanh() 
        self.fc2 = torch.nn.Linear(hidden_layer_size,6) # Output layer
        self.softmax = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        output = self.fc1(x)
        output = self.tanh(output)
        output = self.fc2(output)
        output = self.softmax(output)
        return output

In [16]:
# Function to extract TFIDF vector from train, dev and test set and to run the logistic regression/NN models

from sklearn.feature_extraction.text import TfidfVectorizer

# Gets the tfidf vectors for the train, dev and test set (requires all 3 at the same time to ensure same transformation is applied)
def tf_idf_features(train_cleaned_txt,dev_cleaned_txt,test_cleaned_txt, max_feats,max_df):
    tokenized_doc = train_cleaned_txt
    # tokenized_doc = tokenized_doc['cleaned_text_lst'].apply(lambda x: x.split())

    # remove stop-words

    stop_word_lst = list(_stop_words.ENGLISH_STOP_WORDS)
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_word_lst])

    # de-tokenization
    detokenized_doc_train = []
    for i in range(len(tokenized_doc)):
        t = ' '.join(tokenized_doc.iloc[i])
        detokenized_doc_train.append(t)


    # tokenization dev doc
    tokenized_doc = dev_cleaned_txt
    # tokenized_doc = tokenized_doc['cleaned_text_lst'].apply(lambda x: x.split())

    # remove stop-words

    stop_word_lst = list(_stop_words.ENGLISH_STOP_WORDS)
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_word_lst])

    # de-tokenization
    detokenized_doc_dev = []
    for i in range(len(tokenized_doc)):
        t = ' '.join(tokenized_doc.iloc[i])
        detokenized_doc_dev.append(t)



    # tokenization test doc
    tokenized_doc = test_cleaned_txt
    # tokenized_doc = tokenized_doc['cleaned_text_lst'].apply(lambda x: x.split())

    # remove stop-words

    stop_word_lst = list(_stop_words.ENGLISH_STOP_WORDS)
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_word_lst])

    # de-tokenization
    detokenized_doc_test = []
    for i in range(len(tokenized_doc)):
        t = ' '.join(tokenized_doc.iloc[i])
        detokenized_doc_test.append(t)


    vectorizer = TfidfVectorizer(stop_words='english', 
    max_features= max_feats, # keep top 1000 terms 
    max_df = max_df, 
    smooth_idf=True)

    # need to apply same transform to the dev and test sets
    vectorizer.fit(detokenized_doc_train)
    X_train = vectorizer.transform(detokenized_doc_train)
    X_dev = vectorizer.transform(detokenized_doc_dev)
    X_test = vectorizer.transform(detokenized_doc_test)

    return X_train,X_dev,X_test


def train_nn_model_and_return_accuracies(x_evaluation_set,y_evaluation_set,model_type,epoch,loss_val_min,num_no_progress,per_class_acc):
# Creating logistic regression model

    input_dim = X_torch_train.shape[1]
    if model_type == 'LR':
        model = LRClassifier(input_dim,6)
    elif model_type == 'NN':
        model = Feed_Forward_Neural_Net(input_dim,500) # Using a hidden layer with 500 neurons (Could optimize, used this for simplicity)
    else:
        print('Invalid model selected')
        return
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Training the model
    model.train()
#     epoch = num_epochs
#     loss_val_min = 100
#     num_no_progress = 0
    for epoch in range(epoch):
        optimizer.zero_grad()
        # Forward pass
        y_pred = model(X_torch_train)
    #         print(y_pred.shape)
        # Compute Loss
    #     print(min(y_pred))
    #         print(y_pred[1])
        loss = criterion(y_pred.squeeze().float(), Y_torch_train.long())
        with torch.no_grad():
            y_pred = model(X_torch_dev)
            correct = 0
            loss_val = criterion(y_pred.squeeze().float(), Y_torch_dev.long())
            if epoch%50 ==0:
                print('Epoch {}: train loss: {}       val loss: {}'.format(epoch, loss.item(),loss_val.item()))
            for i in range(len(Y_torch_dev)):
                pred = torch.argmax(y_pred[i])
                ######################### Change this line is loss function changed
            #             truth = torch.argmax(Y_torch_dev_cv[i])
                truth = Y_torch_dev[i]
                if pred == truth:
                    correct += 1
            acc1 = correct/len(Y_torch_dev)
            #         acc_vals.append(acc1)
            if loss_val < loss_val_min:
                loss_val_min = loss_val
                num_no_progress = 0
            else:
                num_no_progress += 1

            if num_no_progress >= 5:
                break
        # Backward pass
        loss.backward()
        optimizer.step()


    # Evalutate accuracy on evaluation set
    model.eval()
    y_pred = model(x_evaluation_set)
    correct = 0
    for i in range(len(y_evaluation_set)):
        pred = torch.argmax(y_pred[i])
        ######################### Change this line is loss function changed
    #             truth = torch.argmax(y_evaluation_set_cv[i])
        truth = y_evaluation_set[i]
        if pred == truth:
            correct += 1
    acc1 = correct/len(y_evaluation_set)

    y_pred = model(X_torch_train)
    correct = 0
    for i in range(len(Y_torch_train)):
        pred = torch.argmax(y_pred[i])
        ######################### Change this line is loss function changed
    #             truth = torch.argmax(y_evaluation_set_cv[i])
        truth = Y_torch_train[i]
        if pred == truth:
            correct += 1
    acc2 = correct/len(Y_torch_train)

    print('test accuracy is:',acc1)    
    
    if per_class_acc == True: # Print per class accuracies if this boolean is true
        nb_classes = 6

        confusion_matrix = torch.zeros(nb_classes, nb_classes)
        with torch.no_grad():
        #     for i, (inputs, classes) in enumerate(test_loader):
        #     inputs = inputs.to(device)
        #     classes = classes.to(device)
            outputs = model(x_evaluation_set)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(y_evaluation_set.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1

        # Per class accuracy
        print('per class acc:')
        print(confusion_matrix.diag()/confusion_matrix.sum(1))
        print('\n\n\n')
        return acc1,confusion_matrix
    return

# LR, on test set
# train_nn_model_and_return_accuracies(X_torch_test,Y_torch_test,'LR',401,100,0,True)


# # NN on test set
# train_nn_model_and_return_accuracies(X_torch_test,Y_torch_test,'NN',301,100,500,True)


## Training Logistic regression models
#### First determine optimal hyperparameters

In [9]:
# Determining the optimal hyperparameters for "max features" and "max_df" for LR by evaluating accuracies on dev set
for max_feats in(500,1000,2000):
    for min_occurences in(0.3,0.5,0.7):
        print('max feats val =',max_feats, 'max_df =',min_occurences)
        
        # Get TFIDF features
        X_train,X_dev,X_test = tf_idf_features(df_train_category['cleaned_text_lst'],\
                                               df_dev_category['cleaned_text_lst'],\
                                               df_test_category['cleaned_text_lst'],max_feats,min_occurences) # Getting tfid features
        
        # Training and dev labels
        Y_torch_train = torch.FloatTensor(df_train_category['class'].to_numpy())
        Y_torch_dev = torch.FloatTensor(df_dev_category['class'].to_numpy())
        # Y_torch_dev = Y_torch_test

        # Converting everything to tensors
        X_torch_train_content = torch.FloatTensor(X_train.toarray())
        X_torch_dev_content = torch.FloatTensor(X_dev.toarray())
        # X_torch_dev_content = X_torch_test_content

        # Extracting length of the reviews as an additional feature
        review_length_torch_train = torch.FloatTensor(df_train_category['len_normalized'].to_numpy())
        review_length_torch_dev = torch.FloatTensor(df_dev_category['len_normalized'].to_numpy())
        # review_length_torch_dev = review_length_torch_test
        
        # Concatenating the length of the review to the TFIDF vector
        X_torch_train = torch.cat((X_torch_train_content, review_length_torch_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_content, review_length_torch_dev.unsqueeze(1)), 1)
        
        train_nn_model_and_return_accuracies(X_torch_dev,Y_torch_dev,'LR',401,100,0,False)
# x_evaluation_set,y_evaluation_set,model_type,num_epochs,loss_val_min,num_no_progress,per_class_acc

max feats val = 500 max_df = 0.3
Epoch 0: train loss: 1.792738437652588       val loss: 1.7926636934280396
Epoch 50: train loss: 1.4555295705795288       val loss: 1.4967467784881592
Epoch 100: train loss: 1.4190374612808228       val loss: 1.4679877758026123
Epoch 150: train loss: 1.4082634449005127       val loss: 1.4589922428131104
Epoch 200: train loss: 1.4038244485855103       val loss: 1.4551243782043457
Epoch 250: train loss: 1.4017456769943237       val loss: 1.45331609249115
Epoch 300: train loss: 1.400684118270874       val loss: 1.452474594116211
Epoch 350: train loss: 1.4000965356826782       val loss: 1.4521028995513916
Epoch 400: train loss: 1.3997650146484375       val loss: 1.4519506692886353
test accuracy is: 0.417
max feats val = 500 max_df = 0.5
Epoch 0: train loss: 1.791796326637268       val loss: 1.7910701036453247
Epoch 50: train loss: 1.4540984630584717       val loss: 1.495802402496338
Epoch 100: train loss: 1.4165393114089966       val loss: 1.4662984609603882

In [17]:
# Getting dfidf vectors with optimal hyperparameters
best_hyperparam_feats = 1000
best_hyperparam_df = 0.7
X_train,X_dev,X_test = tf_idf_features(df_train_category['cleaned_text_lst'],\
                                               df_dev_category['cleaned_text_lst'],\
                                               df_test_category['cleaned_text_lst'],best_hyperparam_feats,best_hyperparam_df)

In [18]:
# Getting length of summaries and extracting tfidf vector
df_train_category['len_summary'] = df_train_category['summary_cleaned'].map(lambda x:len(x))
df_dev_category['len_summary'] = df_dev_category['summary_cleaned'].map(lambda x:len(x))
df_test_category['len_summary'] = df_test_category['summary_cleaned'].map(lambda x:len(x))

min_len_summaries = df_train_category['len_summary'].min()
max_len_summaries = df_train_category['len_summary'].max()

df_train_category['len_summary_normalized'] = (df_train_category['len_summary']-min_len_summaries)/(max_len_summaries-min_len_summaries)
df_dev_category['len_summary_normalized'] = (df_dev_category['len_summary']-min_len_summaries)/(max_len_summaries-min_len_summaries)
df_test_category['len_summary_normalized'] = (df_test_category['len_summary']-min_len_summaries)/(max_len_summaries-min_len_summaries)

X_train_summary,X_dev_summary,X_test_summary = tf_idf_features(df_train_category['summary_cleaned'],\
                                               df_dev_category['summary_cleaned'],\
                                               df_test_category['summary_cleaned'],1000,0.5)

In [19]:
# Relevant tensors (features will be concatenated based on what's included in the model)
Y_torch_train = torch.FloatTensor(df_train_category['class'].to_numpy())
Y_torch_dev = torch.FloatTensor(df_dev_category['class'].to_numpy())
Y_torch_test = torch.FloatTensor(df_test_category['class'].to_numpy())
# Y_torch_dev = Y_torch_test


X_torch_train_content = torch.FloatTensor(X_train.toarray())
X_torch_dev_content = torch.FloatTensor(X_dev.toarray())
X_torch_test_content = torch.FloatTensor(X_test.toarray())
# X_torch_dev_content = X_torch_test_content

X_torch_train_summary = torch.FloatTensor(X_train_summary.toarray())
X_torch_dev_summary = torch.FloatTensor(X_dev_summary.toarray())
X_torch_test_summary = torch.FloatTensor(X_test_summary.toarray())
# X_torch_dev_summary = X_torch_test_summary

review_length_summary_torch_train = torch.FloatTensor(df_train_category['len_summary_normalized'].to_numpy())
review_length_summary_torch_dev = torch.FloatTensor(df_dev_category['len_summary_normalized'].to_numpy())
review_length_summary_torch_test = torch.FloatTensor(df_test_category['len_summary_normalized'].to_numpy())

X_torch_train_summary = torch.cat((X_torch_train_summary, review_length_summary_torch_train.unsqueeze(1)), 1)
X_torch_dev_summary = torch.cat((X_torch_dev_summary, review_length_summary_torch_dev.unsqueeze(1)), 1)
X_torch_test_summary = torch.cat((X_torch_test_summary, review_length_summary_torch_test.unsqueeze(1)), 1)


review_length_torch_train = torch.FloatTensor(df_train_category['len_normalized'].to_numpy())
review_length_torch_dev = torch.FloatTensor(df_dev_category['len_normalized'].to_numpy())
review_length_torch_test = torch.FloatTensor(df_test_category['len_normalized'].to_numpy())
# review_length_torch_dev = review_length_torch_test

year_train = torch.FloatTensor(df_train_category['review_year_normalized'].to_numpy())
year_dev = torch.FloatTensor(df_dev_category['review_year_normalized'].to_numpy())
year_test = torch.FloatTensor(df_test_category['review_year_normalized'].to_numpy())
# year_dev = year_test

In [20]:
# Selecting the features used by the model and concatenating the relevant ones
def included_feats(included_str):
    if included_str == 'All features':
        X_torch_train = torch.cat((X_torch_train_content, review_length_torch_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_content, review_length_torch_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test_content, review_length_torch_test.unsqueeze(1)), 1)

        X_torch_train = torch.cat((X_torch_train, year_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev, year_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test, year_test.unsqueeze(1)), 1)

        X_torch_train = torch.cat((X_torch_train, X_torch_train_summary), 1)
        X_torch_dev = torch.cat((X_torch_dev, X_torch_dev_summary), 1)
        X_torch_test = torch.cat((X_torch_test, X_torch_test_summary), 1)

    elif included_str == 'Non content only':
        # Summary of review, and the year
        X_torch_train = torch.cat((X_torch_train_summary, year_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_summary, year_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test_summary, year_test.unsqueeze(1)), 1)

    elif included_str == 'Content only':
        # Features that have to do with the content of the review only (the review embedding, as well as the review length)
        X_torch_train = torch.cat((X_torch_train_content, review_length_torch_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_content, review_length_torch_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test_content, review_length_torch_test.unsqueeze(1)), 1)

    elif included_str == 'Content and summary':
        # Features that have to do with the content of the review only (the review embedding, as well as the review length)
        X_torch_train = torch.cat((X_torch_train_content, review_length_torch_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_content, review_length_torch_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test_content, review_length_torch_test.unsqueeze(1)), 1)

        X_torch_train = torch.cat((X_torch_train, X_torch_train_summary), 1)
        X_torch_dev = torch.cat((X_torch_dev, X_torch_dev_summary), 1)
        X_torch_test = torch.cat((X_torch_test, X_torch_test_summary), 1)

    elif included_str == 'Content and year':
        # Features that have to do with the content of the review only (the review embedding, as well as the review length)
        X_torch_train = torch.cat((X_torch_train_content, review_length_torch_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_content, review_length_torch_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test_content, review_length_torch_test.unsqueeze(1)), 1)

        X_torch_train = torch.cat((X_torch_train, year_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev, year_dev.unsqueeze(1)), 1)
        X_torch_test = torch.cat((X_torch_test, year_test.unsqueeze(1)), 1)
    else:
        print('Invalid Input String')
        return -1
        
    return X_torch_train,X_torch_dev,X_torch_test

In [13]:
review_length_torch_train.shape

torch.Size([42000])

### Run the Logistic regression model using the various combinations of features.

In [21]:
# Combinations of features to try
included_strings = ['Content only','Non content only','Content and summary','Content and year','All features']

# Store accuracies and confusion matrices
accs_lr = dict()
conf_mat_lr = dict()

for included_str in included_strings:
    X_torch_train,X_torch_dev,X_torch_test = included_feats(included_str)
    print('-----------------------',included_str,'-----------------------\n')
    acc,cm = train_nn_model_and_return_accuracies(X_torch_test,Y_torch_test,'LR',401,100,0,True)
    accs_lr[included_str] = acc
    conf_mat_lr[included_str] = cm/1000.0


----------------------- Content only -----------------------

Epoch 0: train loss: 1.7917453050613403       val loss: 1.7916069030761719
Epoch 50: train loss: 1.3798258304595947       val loss: 1.4528212547302246
Epoch 100: train loss: 1.3278073072433472       val loss: 1.4263564348220825
Epoch 150: train loss: 1.311108112335205       val loss: 1.4181230068206787
Epoch 200: train loss: 1.3036954402923584       val loss: 1.4145101308822632
Epoch 250: train loss: 1.2999207973480225       val loss: 1.412767767906189
Epoch 300: train loss: 1.2978312969207764       val loss: 1.4119467735290527
Epoch 350: train loss: 1.2966251373291016       val loss: 1.4116170406341553
Epoch 400: train loss: 1.295884609222412       val loss: 1.4115376472473145
test accuracy is: 0.429
per class acc:
tensor([0.4830, 0.4590, 0.3140, 0.3760, 0.4360, 0.5060])




----------------------- Non content only -----------------------

Epoch 0: train loss: 1.79207181930542       val loss: 1.7920410633087158
Epoch 50: tr

In [22]:
display(conf_mat_lr)
accs_lr

{'Content only': 0.000429,
 'Non content only': 0.00032700000000000003,
 'Content and summary': 0.00043766666666666666,
 'Content and year': 0.0004335,
 'All features': 0.00044}

{'Content only': tensor([[483., 177., 135.,  61., 116.,  28.],
         [201., 459.,  85., 154.,  58.,  43.],
         [196.,  89., 314., 135., 165., 101.],
         [ 50., 146., 172., 376.,  72., 184.],
         [ 91.,  71., 117.,  77., 436., 208.],
         [ 35.,  42.,  64., 144., 209., 506.]]),
 'Non content only': tensor([[457., 155., 161.,  75.,  94.,  58.],
         [269., 343., 136., 104.,  79.,  69.],
         [211.,  94., 291., 149., 124., 131.],
         [125., 150., 203., 230., 118., 174.],
         [165.,  82., 176.,  84., 276., 217.],
         [113.,  67., 140., 121., 194., 365.]]),
 'Content and summary': tensor([[502., 166., 155.,  61.,  86.,  30.],
         [211., 474.,  89., 122.,  59.,  45.],
         [184.,  85., 324., 160., 162.,  85.],
         [ 49., 134., 197., 356.,  82., 182.],
         [ 91.,  58., 119.,  85., 462., 185.],
         [ 31.,  34.,  79., 132., 216., 508.]]),
 'Content and year': tensor([[499., 174., 131.,  57., 110.,  29.],
         [198., 463., 

In [29]:
# Saving accuracies and confusion matrices for all models to compare later in the analysis
accs_all_classifiers['lr'] = accs_lr
conf_mats_all_classifiers['lr'] = conf_mat_lr

## Training 1-hidden layer NN models
#### First determine optimal hyperparameters

In [19]:
# Determining the optimal hyperparameters for "max features" and "max_df" for LR by evaluating accuracies on dev set
for max_feats in(500,1000,2000):
    for min_occurences in(0.3,0.5,0.7):
        print('max feats val =',max_feats, 'max_df =',min_occurences)
        
        # Get TFIDF features
        X_train,X_dev,X_test = tf_idf_features(df_train_category['cleaned_text_lst'],\
                                               df_dev_category['cleaned_text_lst'],\
                                               df_test_category['cleaned_text_lst'],max_feats,min_occurences) # Getting tfid features
        
        # Training and dev labels
        Y_torch_train = torch.FloatTensor(df_train_category['class'].to_numpy())
        Y_torch_dev = torch.FloatTensor(df_dev_category['class'].to_numpy())
        # Y_torch_dev = Y_torch_test

        # Converting everything to tensors
        X_torch_train_content = torch.FloatTensor(X_train.toarray())
        X_torch_dev_content = torch.FloatTensor(X_dev.toarray())
        # X_torch_dev_content = X_torch_test_content

        # Extracting length of the reviews as an additional feature
        review_length_torch_train = torch.FloatTensor(df_train_category['len_normalized'].to_numpy())
        review_length_torch_dev = torch.FloatTensor(df_dev_category['len_normalized'].to_numpy())
        # review_length_torch_dev = review_length_torch_test
        
        # Concatenating the length of the review to the TFIDF vector
        X_torch_train = torch.cat((X_torch_train_content, review_length_torch_train.unsqueeze(1)), 1)
        X_torch_dev = torch.cat((X_torch_dev_content, review_length_torch_dev.unsqueeze(1)), 1)
        
        train_nn_model_and_return_accuracies(X_torch_dev,Y_torch_dev,'NN',301,100,500,False)
# x_evaluation_set,y_evaluation_set,model_type,num_epochs,loss_val_min,num_no_progress,per_class_acc

max feats val = 500 max_df = 0.3
Epoch 0: train loss: 1.7918574810028076       val loss: 1.7917752265930176
test accuracy is: 0.4
max feats val = 500 max_df = 0.5
Epoch 0: train loss: 1.7917269468307495       val loss: 1.7917548418045044
test accuracy is: 0.4048333333333333
max feats val = 500 max_df = 0.7
Epoch 0: train loss: 1.791793704032898       val loss: 1.791735053062439
test accuracy is: 0.406
max feats val = 1000 max_df = 0.3
Epoch 0: train loss: 1.7916555404663086       val loss: 1.7917845249176025
test accuracy is: 0.4266666666666667
max feats val = 1000 max_df = 0.5
Epoch 0: train loss: 1.791955590248108       val loss: 1.7918492555618286
test accuracy is: 0.42533333333333334
max feats val = 1000 max_df = 0.7
Epoch 0: train loss: 1.791745901107788       val loss: 1.7917332649230957
test accuracy is: 0.42483333333333334
max feats val = 2000 max_df = 0.3
Epoch 0: train loss: 1.791524052619934       val loss: 1.7917063236236572
test accuracy is: 0.43433333333333335
max feats v

In [23]:
# Getting dfidf vectors with optimal hyperparameters
best_hyperparam_feats = 1000
best_hyperparam_df = 0.7
X_train,X_dev,X_test = tf_idf_features(df_train_category['cleaned_text_lst'],\
                                               df_dev_category['cleaned_text_lst'],\
                                               df_test_category['cleaned_text_lst'],best_hyperparam_feats,best_hyperparam_df)
# Converting training data to tensors
X_torch_train_content = torch.FloatTensor(X_train.toarray())
X_torch_dev_content = torch.FloatTensor(X_dev.toarray())
X_torch_test_content = torch.FloatTensor(X_test.toarray())

### Run the the Neural network using the various combinations of features.

In [24]:
# Combinations of features to try
included_strings = ['Content only','Non content only','Content and summary','Content and year','All features']

# Store accuracies and confusion matrices
accs_NN = dict()
conf_mat_NN = dict()

for included_str in included_strings:
    X_torch_train,X_torch_dev,X_torch_test = included_feats(included_str)
    print('-----------------------',included_str,'-----------------------\n')
    acc,cm = train_nn_model_and_return_accuracies(X_torch_test,Y_torch_test,'NN',301,100,500,True)
    accs_NN[included_str] = acc
    conf_mat_NN[included_str] = cm/1000.0



----------------------- Content only -----------------------

Epoch 0: train loss: 1.7917693853378296       val loss: 1.7917897701263428
test accuracy is: 0.41983333333333334
per class acc:
tensor([0.4760, 0.4670, 0.2950, 0.3690, 0.4210, 0.4910])




----------------------- Non content only -----------------------

Epoch 0: train loss: 1.7918318510055542       val loss: 1.7917348146438599
test accuracy is: 0.323
per class acc:
tensor([0.4180, 0.3440, 0.2610, 0.2450, 0.3090, 0.3610])




----------------------- Content and summary -----------------------

Epoch 0: train loss: 1.7919135093688965       val loss: 1.7917280197143555
test accuracy is: 0.43266666666666664
per class acc:
tensor([0.5260, 0.4480, 0.3230, 0.3610, 0.4020, 0.5360])




----------------------- Content and year -----------------------

Epoch 0: train loss: 1.7916138172149658       val loss: 1.7917559146881104
test accuracy is: 0.418
per class acc:
tensor([0.4740, 0.4650, 0.2910, 0.3710, 0.3930, 0.5140])




---------

In [65]:
 # Creating Neural Network with 1 hidden layer

accs_NN = dict()
conf_mat_NN = dict()
for included_str in included_strings:
    X_torch_train,X_torch_dev,X_torch_test = included_feats(included_str)
    print('-----------------------',included_str,'-----------------------\n')    

    input_dim = X_torch_train.shape[1]
    model = Feed_Forward_Neural_Net(input_dim,500)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)


    # Training the model
    model.train()
    epoch = 301
    loss_val_min = 100
    num_no_progress = 500
    for epoch in range(epoch):
        optimizer.zero_grad()
        # Forward pass
        y_pred = model(X_torch_train)
    #         print(y_pred.shape)
        # Compute Loss
    #     print(min(y_pred))
    #         print(y_pred[1])
        loss = criterion(y_pred.squeeze().float(), Y_torch_train.long())
        with torch.no_grad():
            y_pred = model(X_torch_dev)
            correct = 0
            loss_val = criterion(y_pred.squeeze().float(), Y_torch_dev.long())
            if epoch % 10 == 0:
                print('Epoch {}: train loss: {}       val loss: {}'.format(epoch, loss.item(),loss_val.item()))
            for i in range(len(Y_torch_dev)):
                pred = torch.argmax(y_pred[i])
                ######################### Change this line is loss function changed
            #             truth = torch.argmax(Y_torch_test_cv[i])
                truth = Y_torch_dev[i]
                if pred == truth:
                    correct += 1
            acc1 = correct/len(Y_torch_dev)
            #         acc_vals.append(acc1)
            if loss_val < loss_val_min:
                loss_val_min = loss_val
                num_no_progress = 0
            else:
                num_no_progress += 1

            if num_no_progress >= 5:
                break
        # Backward pass
        loss.backward()
        optimizer.step()


    # Evalutate accuracy on test set
    model.eval()
    y_pred = model(X_torch_test)
    correct = 0
    for i in range(len(Y_torch_test)):
        pred = torch.argmax(y_pred[i])
        ######################### Change this line is loss function changed
    #             truth = torch.argmax(Y_torch_test_cv[i])
        truth = Y_torch_test[i]
        if pred == truth:
            correct += 1
    acc1 = correct/len(Y_torch_test)

    y_pred = model(X_torch_train)
    correct = 0
    for i in range(len(Y_torch_train)):
        pred = torch.argmax(y_pred[i])
        ######################### Change this line is loss function changed
    #             truth = torch.argmax(Y_torch_test_cv[i])
        truth = Y_torch_train[i]
        if pred == truth:
            correct += 1
    acc2 = correct/len(Y_torch_train)

    accs_NN[included_str] = acc1

    
    print('test accuracy is:',acc1)
    
    nb_classes = 6

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
    #     for i, (inputs, classes) in enumerate(test_loader):
    #     inputs = inputs.to(device)
    #     classes = classes.to(device)
        outputs = model(X_torch_test)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(Y_torch_test.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    conf_mat_NN[included_str] = confusion_matrix/1000
    # Per class accuracy
    print('Per Class Accuracy:')
    print(confusion_matrix.diag()/confusion_matrix.sum(1))
    print('\n\n\n')

----------------------- Content only -----------------------

Epoch 0: train loss: 1.7920063734054565       val loss: 1.7917832136154175
Epoch 10: train loss: 1.6427983045578003       val loss: 1.679885745048523
test accuracy is: 0.4185
Per Class Accuracy:
tensor([0.4790, 0.4970, 0.3230, 0.3160, 0.4060, 0.4900])




----------------------- Non content only -----------------------

Epoch 0: train loss: 1.7916207313537598       val loss: 1.7917343378067017
Epoch 10: train loss: 1.6949483156204224       val loss: 1.7261940240859985
test accuracy is: 0.3055
Per Class Accuracy:
tensor([0.4790, 0.3530, 0.2030, 0.1900, 0.2730, 0.3350])




----------------------- Content and summary -----------------------

Epoch 0: train loss: 1.7917296886444092       val loss: 1.7917546033859253
Epoch 10: train loss: 1.6170746088027954       val loss: 1.6740939617156982
test accuracy is: 0.44083333333333335
Per Class Accuracy:
tensor([0.5080, 0.4930, 0.3080, 0.3740, 0.4390, 0.5230])




--------------------

In [30]:
# Saving accuracies and confusion matrices for later analysis
accs_all_classifiers['nn'] = accs_NN
conf_mats_all_classifiers['nn'] = conf_mat_NN

review_by_category_accs['Moviestv'] = accs_all_classifiers
review_by_category_confusion_matrix['Moviestv'] = conf_mats_all_classifiers

In [35]:
review_by_category_accs

{'Moviestv': {'nn': {'Content only': tensor([[476., 172., 143.,  72.,  98.,  39.],
           [184., 467., 103., 133.,  63.,  50.],
           [194., 101., 295., 152., 154., 104.],
           [ 55., 150., 175., 369.,  76., 175.],
           [ 92.,  85., 113.,  95., 421., 194.],
           [ 38.,  48.,  70., 154., 199., 491.]]),
   'Non content only': tensor([[418., 159., 156.,  72., 122.,  73.],
           [258., 344., 119., 113.,  94.,  72.],
           [182.,  94., 261., 161., 158., 144.],
           [123., 143., 192., 245., 122., 175.],
           [130.,  74., 145., 114., 309., 228.],
           [ 93.,  60., 128., 141., 217., 361.]]),
   'Content and summary': tensor([[526., 156., 138.,  67.,  76.,  37.],
           [220., 448., 101., 131.,  47.,  53.],
           [187.,  81., 323., 178., 135.,  96.],
           [ 51., 128., 193., 361.,  78., 189.],
           [ 93.,  52., 118.,  89., 402., 246.],
           [ 32.,  35.,  76., 142., 179., 536.]]),
   'Content and year': tensor([[474

In [ ]:
from Training_Models import train_all_models_and_return_results
accs_all_classifiers,conf_mats_all_classifiers = train_all_models_and_return_results()

review_by_category_accs['Books'] = accs_all_classifiers
review_by_category_confusion_matrix['Books'] = conf_mats_all_classifiers

In [40]:
review_by_category_accs['Books'] = accs_all_classifiers
review_by_category_confusion_matrix['Books'] = conf_mats_all_classifiers

In [41]:
review_by_category_accs

{'Moviestv': {'nn': {'Content only': tensor([[476., 172., 143.,  72.,  98.,  39.],
           [184., 467., 103., 133.,  63.,  50.],
           [194., 101., 295., 152., 154., 104.],
           [ 55., 150., 175., 369.,  76., 175.],
           [ 92.,  85., 113.,  95., 421., 194.],
           [ 38.,  48.,  70., 154., 199., 491.]]),
   'Non content only': tensor([[418., 159., 156.,  72., 122.,  73.],
           [258., 344., 119., 113.,  94.,  72.],
           [182.,  94., 261., 161., 158., 144.],
           [123., 143., 192., 245., 122., 175.],
           [130.,  74., 145., 114., 309., 228.],
           [ 93.,  60., 128., 141., 217., 361.]]),
   'Content and summary': tensor([[526., 156., 138.,  67.,  76.,  37.],
           [220., 448., 101., 131.,  47.,  53.],
           [187.,  81., 323., 178., 135.,  96.],
           [ 51., 128., 193., 361.,  78., 189.],
           [ 93.,  52., 118.,  89., 402., 246.],
           [ 32.,  35.,  76., 142., 179., 536.]]),
   'Content and year': tensor([[474